# 安裝pkgo專案所需要的模組庫及匯入套件

In [ ]:
!apt -qq install proj-bin libproj-dev libgeos-dev
!apt-get install libgeos-3.5.0 #需指定使用這一版
!apt-get install libgeos-dev
!pip install https://github.com/matplotlib/basemap/archive/master.zip
!pip install pyproj==1.9.6 #需指定使用這一版

<hr/>

![alt text](https://gitlab.aiacademy.tw/tclin/at091-DL/raw/master/pic/ML_topic.png)

<hr/>

![alt text](https://gitlab.aiacademy.tw/tclin/at091-DL/raw/master/pic/ML-pipline.png)

<hr/>

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import os
import time
import numpy as np
import pandas as pd
from google.colab import files
from mpl_toolkits.basemap import Basemap
from matplotlib import animation
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

import requests

# 下載且準備pkgo專案所需的資料集

In [ ]:
# 設定 GitHub 下載網址
data_file = '300k.csv.bz2'
dataset_url = 'https://github.com/hdyen/hands-on-dl/raw/master/PKGO/dataset/'
data_url = '{}{}'.format(dataset_url, data_file)

if not os.path.exists(data_file):
    try:
        r = requests.get(data_url)
        # 將從 GitHub 下戴的資料集寫入 colab 本地端
        with open(data_file, 'wb') as f:
            f.write(r.content)
    except:
        # 如果無法從 GitHub 下戴資料，則自行上傳 300k.csv.bz2
        print('請將本機端的資料 300k.csv.bz2 上傳到 colab 環境')
        uploaded = files.upload()

#將下載好的資料內容存放到變數pkgo當中
pkgo = pd.read_csv(data_file, low_memory=False)

# 原始資料一覽
#### 先觀察pkgo資料的部分欄位

In [ ]:
pkgo.shape #藉由shape得知，pkgo為一個二維矩陣的資料

In [ ]:
print('pkgo 一共有 {} 筆資料樣本，{} 個資料欄位'.format(pkgo.shape[0], pkgo.shape[1]))

In [ ]:
pkgo.head(1) #未定義前會省略部分columns的顯示 (原有208個欄位，只顯示20個欄位)

In [ ]:
print('before set options :',pd.get_option('display.max_columns'))

#自定義選項，使columns能夠全盤顯示
pd.set_option('display.max_columns',pkgo.shape[1])
print('after set options :',pd.get_option('display.max_columns'))
pkgo.head(1)

In [ ]:
pkgo.columns.values #在pkgo的csv檔中，所有Features的值

## [Pkgo's Features detail]


https://drive.google.com/file/d/1DdEfk2NF1aks-91ulXIMkGa0vhoh4ExO/view

![alt text](https://gitlab.aiacademy.tw/tclin/at091-DL/raw/master/pic/Data_Field_Overview.png)

<hr/>

In [ ]:
# 取部分Features來觀察數值
view_list = ['latitude', 'longitude', 'appearedLocalTime',
             'closeToWater', 'terrainType',
             'temperature', 'windSpeed', 'class']
pkgo[view_list].head(10)

## 寶可夢出現的位置

In [ ]:
pkgo_location = pkgo[['city', 'latitude', 'longitude']].head(10) # 利用city和經緯度的cloumns，來去觀察寶可夢的城市位置。

In [ ]:
pkgo_location

In [ ]:
pkgo['city'].value_counts() #統計City中，各個資料樣本筆數。(從此得出，紐約市的寶可夢是最多的！）

## 只看表格不是那麼清楚，從地圖來了解寶可夢出沒的地方
由於pyproj 2.0.1 的問題，在使用basemap前，必須將其降級至pyproj 1.9.6，於第一行下載套件已執行。

In [ ]:
def map_plot(pkgo):
    # https://matplotlib.org/api/_as_gen/matplotlib.pyplot.figure.html - pyplot.figure
    plt.figure(1, figsize=(20,10))
    # https://basemaptutorial.readthedocs.io/en/latest/plotting_data.html - Basemap
    m1 = Basemap(projection='merc',
                 llcrnrlat=-60,
                 urcrnrlat=65,
                 llcrnrlon=-180,
                 urcrnrlon=180,
                 lat_ts=0,
                 resolution='c')

    m1.fillcontinents(color='#191919',lake_color='#000000') # 設置地圖內部顏色：dark grey (land), black (lakes)
    m1.drawmapboundary(fill_color='#000000')        # 設置地圖背景顏色：black (background)
    m1.drawcountries(linewidth=0.3, color="y")       # 設置國家界線顏色：thin color-y line for (country borders)

    # 將data繪製至地圖上
    x, y = m1(pkgo.longitude.tolist(), pkgo.latitude.tolist())
    m1.scatter(x,y, s=3, c="#99ccff", lw=0, alpha=1, zorder=5)
    plt.title("Pokemon activity")
    plt.show()

### 試一下，來看看台北的Pokemon

In [ ]:
pd.DataFrame(pkgo['city'].value_counts()).loc["Taipei",:]

In [ ]:
map_plot(pkgo[pkgo.city=="Taipei"])

## 我們把範圍縮小到紐約

In [ ]:
pd.DataFrame(pkgo['city'].value_counts()).loc["New_York",:]

In [ ]:
pkgo = pkgo[pkgo.city=='New_York']
map_plot(pkgo)

## 讓我們從經緯度的分佈來看看
### 經緯度平面圖

In [ ]:
sns.jointplot(x='longitude', y='latitude', data=pkgo) #散點圖繪製，x、y軸設置經緯度中pkgo的資料。

### 設定經緯度範圍，將資料縮限在美東

In [ ]:
pkgo = pkgo[ (-90 < pkgo['longitude']) & (pkgo['longitude'] < -50) & 
            (20 < pkgo['latitude']) & (pkgo['latitude']< 50)]

map_plot(pkgo)

### 挑選五隻寶可夢作為分類目標
#### 留下 小火龍(No.04)、走路草(No.43)、猴怪(No.56)、口呆花(No.71)、大鉗蟹(No.98)

In [ ]:
# 小火龍(No.04)、走路草(No.43)、猴怪(No.56)、口呆花(No.71)、大鉗蟹(No.98)
pokemons = [pkgo[pkgo['class']==(i)] for i in [4, 43, 56, 71, 98]]
pkgo = pd.concat(pokemons)

map_plot(pkgo)

## 處理名目資料 (Nominal variables) - 資料前處理
### 標籤編碼：手動編碼
#### 以appearedDayOfWeek為例，處理前：

In [ ]:
col_list = ['latitude', 'longitude', 
            'appearedDayOfWeek', 'class']
pkgo[col_list].iloc[[i for i in range(1,1000,150)]] #取出[1,151,301,451,601,751,901]筆資料 - 介於1~1000，間隔150

#### 處理前，appearedDayOfWeek的值如下

In [ ]:
set(pkgo['appearedDayOfWeek']) # set 顯示無序不重複元素集，可進行關係測試，刪除重複數據，還可以計算交集、差集、並集等。

#### 手動編碼

In [ ]:
label_map = {'dummy_day': 0, 'Monday': 1, 'Tuesday': 2, 'Wednesday': 3,
             'Thursday': 4, 'Friday': 5, 'Saturday': 6, 'Sunday': 7}

#將編碼後的label map存至pkgo['appearedDayOfWeek']中。
pkgo['appearedDayOfWeek'] = pkgo['appearedDayOfWeek'].map(label_map)

#### 以appearedDayOfWeek為例，處理後
**處理後，appearedDayOfWeek的值如下**

In [ ]:
set(pkgo['appearedDayOfWeek']) # set 顯示無序不重複元素集，可進行關係測試，刪除重複數據，還可以計算交集、差集、並集等。

In [ ]:
pkgo[col_list].iloc[[i for i in range(1, 1000, 150)]]

### 標籤編碼：自動編碼
#### 以weather為例，處理前

In [ ]:
col_list = ['latitude', 'longitude', 'weather', 'class']
pkgo[col_list].iloc[[i for i in range(1,1500,200)]] #取出[1,201,401,601,801,1001,1201,1401]筆資料 - 介於1~1500，間隔200

In [ ]:
pkgo['weather'].unique() #unique 函數去除其中重複的元素，並按元素由大到小返回一個新的無元素重複的元組或者列表

In [ ]:
#利用scikitlearn的preprocessing做LabelEncoder
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

#擬合數據後，再將Encoder好的數據轉換上去
le.fit(pkgo['weather'].unique())
pkgo['weather'] = le.transform(pkgo['weather'].values)

In [ ]:
pkgo[col_list].iloc[[i for i in range(1,1500,200)]]

In [ ]:
set(pkgo['weather']) #用set觀察是否都已完成LabelEncoder

In [ ]:
pkgo.shape

**示範欄位的布林值轉0跟1的數值**

In [ ]:
pkgo['urban'].head(5)

In [ ]:
pkgo['urban'] = pkgo['urban'].astype(int)

In [ ]:
pkgo['urban'].head(5)

# 關於feature Re-scale

![alt text](https://gitlab.aiacademy.tw/tclin/at091-DL/raw/master/pic/Re-scale.png)

**示範如何做rescale: 將值域轉成0~1之間**

In [ ]:
pkgo['pressure'].head(5)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
#feature scale normalize
scaler = MinMaxScaler()
pkgo['pressure'] = scaler.fit_transform(pkgo['pressure'].values.reshape(-1, 1))

In [ ]:
pkgo['pressure'].head(5)

In [ ]:
pkgo['pressure'].max(), pkgo['pressure'].min()

**轉換時的需注意陣列的形狀需轉為(多列,一欄)的形式: 記得要reshape(-1,1)**

In [ ]:
test_pop = scaler.fit_transform(pkgo['population_density'].values)